### 9-1. 프로젝트 : 모든 장르 간 편향성 측정해 보기


In [1]:
import konlpy
import gensim
import sklearn
import seaborn

print(konlpy.__version__)
print(gensim.__version__)
print(sklearn.__version__)
print(seaborn.__version__)

Matplotlib is building the font cache; this may take a moment.


0.5.2
4.1.2
1.0
0.11.2


In [2]:
import os

with open(os.getenv('HOME')+'/aiffel/weat/synopsis.txt', 'r') as file:
    for i in range(5):
        print(file.readline(), end='')

사운드 엔지니어 상우(유지태 분)는 치매에 걸린 할머니(백성희 분)와
 젊은 시절 상처한 한 아버지(박인환 분), 고모(신신애 분)와 함께 살고 있다.
 어느 겨울 그는 지방 방송국 라디오 PD 은수(이영애 분)를 만난다.
 자연의 소리를 채집해 틀어주는 라디오 프로그램을 준비하는 은수는 상우와 녹음 여행을 떠난다.
 자연스레 가까워지는 두 사람은 어느 날, 은수의 아파트에서 밤을 보낸다.


### STEP 1. 형태소 분석기를 이용하여 품사가 명사인 경우 해당 단어를 추출하기 with synopsis.txt  


In [76]:
from konlpy.tag import Okt
okt = Okt()
tokenized = []
with open(os.getenv('HOME')+'/aiffel/weat/synopsis.txt', 'r') as file:
    while True:
        line = file.readline()
        if not line: break
        words = okt.pos(line, stem=True, norm=True)
        res = []
        for w in words:
            if w[1] in ["Noun"]:      # "Adjective", "Verb" 등을 포함할 수도 있습니다.
                res.append(w[0])   
        tokenized.append(res)

In [77]:
print(len(tokenized))

71156


### STEP 2. 추출된 결과로 embedding model 만들기  (Word2Vec)

In [78]:
from gensim.models import Word2Vec

model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)  
model.wv.most_similar(positive=['영화'])

[('작품', 0.8919191956520081),
 ('다큐멘터리', 0.8583681583404541),
 ('드라마', 0.8124423623085022),
 ('영화로', 0.8040297627449036),
 ('코미디', 0.7889005541801453),
 ('형식', 0.7784495949745178),
 ('주제', 0.7737573385238647),
 ('소재', 0.77344810962677),
 ('설정', 0.7732013463973999),
 ('스토리', 0.7731793522834778)]

### STEP 3. target, attribute 단어 셋 만들기     
이전 스텝에서는 TF-IDF를 사용해서 단어 셋을 만들었습니다. 이 방법으로도 어느 정도는 대표 단어를 잘 선정할 수 있습니다. 그러나 TF-IDF가 높은 단어를 골랐음에도 불구하고 중복되는 단어가 발생하는 문제가 있었습니다. 개념축을 표현하는 단어가 제대로 선정되지 않은 것은 WEAT 계산 결과에 악영향을 미칩니다.   

TF-IDF를 적용했을 때의 문제점이 무엇인지 지적 가능하다면 그 문제점을 지적하고 스스로 방법을 개선하여 대표 단어 셋을 구축해 보기 바랍니다. TF-IDF 방식을 쓰더라도 중복된 단어를 잘 제거하면 여전히 유용한 방식이 될 수 있습니다

> 기존 노드에서 진행하였던 방식(우선 상위 100개의 단어들 중 중복되는 단어를 제외)는 각 카테고리에 적합한 단어를 중복의 이유로 제거할 수 있다는 문제가 있다. 예를들어 '아버지'는 Family에 굉장히 적합한 단어로 사료되지만 드라마 같은 카테고리에도 중복되어 사용되므로 제거된다. 이러한 제거 방식은 각 카테고리의 적절한 대표 단어 선정을 저해할 가능성이 있다.

>Solution) 공통 단어라도 특정 장르에서 더 중요한 단어는 유지 (중복 단어중, 특정 카테고리에서 TF-IDF 값이 높은 경우 그 카테고리에서 유지)

In [108]:
# genre_txt = ['synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt', 
#              'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt', 
#              'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt', 
#              'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt', 
#              'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt']
# genre_name = ['SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)',
#          '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지'

In [79]:
# 시간상 '가족'& '드라마'로 비교
genre_txt = [ 'synopsis_family.txt', 'synopsis_drama.txt']
genre_name = ['가족',  '드라마']

In [80]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from konlpy.tag import Okt

def read_token(file_name):
    okt = Okt()
    result = []
    with open(os.getenv('HOME')+'/aiffel/weat/'+file_name, 'r') as fread: 
        print(file_name, '파일을 읽고 있습니다.')
        while True:
            line = fread.readline() 
            if not line: break 
            tokenlist = okt.pos(line, stem=True, norm=True) 
            for word in tokenlist:
                if word[1] in ["Noun"]:#, "Adjective", "Verb"]:
                    result.append((word[0])) 
    return ' '.join(result)

In [81]:
genre = []
for file_name in genre_txt:
    genre.append(read_token(file_name))

synopsis_family.txt 파일을 읽고 있습니다.
synopsis_drama.txt 파일을 읽고 있습니다.


In [82]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(genre) # 11개( SF & Family & show ... & drama)를 숫자로 변환

print(X.shape)

(2, 24276)


In [10]:
print(X[1]) # doc Idx, word idx, TF-IDF score 

  (0, 22154)	0.00012912989604844745
  (0, 21084)	0.00012912989604844745
  (0, 12586)	0.0002582597920968949
  (0, 21085)	0.00038738968814534233
  (0, 17913)	0.0005165195841937898
  (0, 6810)	0.00012912989604844745
  (0, 11142)	0.0002582597920968949
  (0, 20458)	0.0005165195841937898
  (0, 7707)	0.0002582597920968949
  (0, 12522)	0.00012912989604844745
  (0, 16334)	0.00012912989604844745
  (0, 13127)	0.00012912989604844745
  (0, 20761)	0.00012912989604844745
  (0, 21588)	0.0002582597920968949
  (0, 9399)	0.00012912989604844745
  (0, 20104)	0.00012912989604844745
  (0, 2751)	0.00012912989604844745
  (0, 4364)	0.00012912989604844745
  (0, 19269)	0.00012912989604844745
  (0, 2870)	0.00012912989604844745
  (0, 14564)	0.00012912989604844745
  (0, 19405)	0.00012912989604844745
  (0, 22466)	0.00012912989604844745
  (0, 12948)	0.00012912989604844745
  (0, 13008)	0.00038738968814534233
  :	:
  (0, 9571)	0.20920389386122448
  (0, 10742)	0.0446522145000242
  (0, 15748)	0.0045019722438296005
  (0, 1

In [83]:
m1 = X[0].tocoo()   # art를 TF-IDF로 표현한 sparse matrix를 가져옵니다. 
m2 = X[1].tocoo()   # gen을 TF-IDF로 표현한 sparse matrix를 가져옵니다. 

w1 = [[i, j] for i, j in zip(m1.col, m1.data)]
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 
w2.sort(key=lambda x: x[1], reverse=True)   #gen을 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 

print('Family 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')
    
print('Drama 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

Family 대표하는 단어들:
엄마, 영화제, 아빠, 가족, 자신, 위해, 친구, 아버지, 시작, 그녀, 아들, 마을, 국제, 사람, 아이, 사랑, 할머니, 학교, 세상, 소년, 이야기, 가장, 어머니, 아주르, 소녀, 대한, 모두, 아내, 사건, 사이, 마음, 혼자, 낙타, 서울, 모험, 위기, 다시, 과연, 결심, 씨제이, 엠마, 할아버지, 서로, 미아, 하나, 작품, 영화, 발견, 한편, 동구, 이자, 고양이, 기억, 도시, 시골, 크리스마스, 단편, 다른, 생각, 시간, 소식, 때문, 간다, 부산, 사실, 손녀, 통해, 요정, 인형, 도움, 인간, 상황, 모든, 해나, 케이시, 편지, 인생, 테리, 순간, 매일, 스튜어트, 여행, 번개, 동생, 아기, 회사, 펠리칸, 슈이트, 점점, 선물, 부부, 가지, 인도, 마르, 아스, 부모님, 동안, 모습, 판타스틱, 부천, 

Drama 대표하는 단어들:
자신, 영화제, 그녀, 사람, 사랑, 위해, 영화, 시작, 국제, 남자, 친구, 이야기, 아버지, 엄마, 가족, 여자, 대한, 아들, 아이, 마음, 단편, 서울, 남편, 서로, 시간, 소녀, 다른, 세상, 감독, 모든, 연출, 다시, 아내, 마을, 사이, 사건, 관계, 생각, 작품, 사실, 모습, 통해, 소년, 때문, 하나, 어머니, 학교, 생활, 간다, 점점, 결혼, 우리, 발견, 대해, 인생, 순간, 여행, 여성, 죽음, 부문, 기억, 상황, 세계, 현실, 가장, 일상, 부산, 독립, 감정, 마지막, 과연, 모두, 과거, 의도, 가지, 한편, 경쟁, 하루, 음악, 배우, 이제, 할머니, 연인, 한국, 아빠, 결심, 동안, 이자, 미국, 전쟁, 사고, 주인공, 동생, 처음, 시절, 도시, 사회, 비밀, 명의, 상처, 

### 중복단어 모두 제거시

In [84]:
n = 15
w1_, w2_ = [], []
for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break 

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break

In [85]:
print(target_art)
print(target_gen)

['아주르', '혼자', '낙타', '모험', '위기', '씨제이', '엠마', '할아버지', '미아', '동구', '고양이', '시골', '크리스마스', '소식', '손녀']
['남자', '여자', '남편', '감독', '연출', '관계', '생활', '결혼', '우리', '대해', '여성', '죽음', '부문', '세계', '현실']


### IDF Score가 높은 곳에 남길시

In [86]:
m_list = []  
length = len(genre_txt)
for i in range(length):
    m_list.append(X[i].tocoo())
len(m_list)

w_list = []  
for i in range(length):
    w_list.append([[i, j] for i, j in zip(m_list[i].col, m_list[i].data)])
len(w_list)

2

In [87]:
len(w_list[0])

2470

In [88]:
new_w1_, new_w2_ = {}, {}

for i in range(100):
    new_w1_[vectorizer.get_feature_names_out()[w1[i][0]]] = w1[i][1]  # word and its weight for w1
    new_w2_[vectorizer.get_feature_names_out()[w2[i][0]]] = w2[i][1]  # word and its weight for w2

In [107]:
threshold = 0.01  # TF-IDF 점수 차이가 0.01 이상일 경우
                  # 중복을 어느정도 허용하고 싶으면 threshold 증가
remove_from_art = set()
remove_from_gen = set()

duplicated_words = set(new_w1_.keys()) & set(new_w2_.keys())

In [91]:
print('Family, Drama 간 중복되는 단어들')
print(duplicated_words)

Family, Drama 간 중복되는 단어들
{'모습', '친구', '가지', '가족', '사실', '다시', '간다', '도시', '결심', '부산', '동생', '다른', '영화제', '점점', '단편', '발견', '마을', '어머니', '소년', '인생', '마음', '여행', '학교', '기억', '사랑', '아빠', '순간', '사건', '아들', '한편', '사이', '위해', '하나', '아내', '동안', '국제', '가장', '때문', '모든', '이자', '서로', '작품', '시간', '그녀', '이야기', '아버지', '생각', '엄마', '시작', '자신', '영화', '서울', '할머니', '과연', '소녀', '상황', '모두', '아이', '대한', '사람', '통해', '세상'}


In [92]:
# 중복시 TF-IDF 계산하여 제거 단어 선정
for w in common_words:
    tfidf_art = new_w1_.get(w, 0)
    tfidf_gen = new_w2_.get(w, 0)

    if tfidf_art > tfidf_gen + threshold:
        remove_from_gen.add(w)  
    elif tfidf_gen > tfidf_art + threshold:
        remove_from_art.add(w) 

# 결과 출력
print("Family 제거 단어:", remove_from_art)
print('====================')
print("Drama 제거 단어:", remove_from_gen)


Family 제거 단어: {'모습', '사실', '다른', '영화제', '점점', '단편', '마음', '사랑', '국제', '때문', '모든', '서로', '시간', '그녀', '이야기', '생각', '시작', '자신', '영화', '서울', '대한', '사람', '통해'}
Drama 제거 단어: {'친구', '가족', '도시', '결심', '마을', '어머니', '소년', '학교', '아빠', '한편', '아들', '위해', '가장', '이자', '아버지', '엄마', '할머니', '과연', '모두', '아이', '세상'}


In [98]:
# Family 상위 15개중 제거 단어에 포함된 갯수
w1.sort(key=lambda x: x[1], reverse=True)
cnt = 0
print('Family 상위 15개중에서 제거 단어 수')
for i in range(15):
    temp = vectorizer.get_feature_names()[w1[i][0]]
    if temp in remove_from_art:
        print(temp)
        cnt += 1
print(cnt,'개')

Family 상위 15개중에서 중복 단어 수
영화제
자신
시작
그녀
국제
사람
6 개


In [103]:
# Drama 상위 15개중 제거 단어에 포함된 갯수
w2.sort(key=lambda x: x[1], reverse=True)
cnt = 0
print('Family 상위 15개중에서 제거 단어 수')
for i in range(15):
    temp = vectorizer.get_feature_names()[w2[i][0]]
    if temp in remove_from_gen:
        print(temp)
        cnt += 1
print(cnt,'개')

Family 상위 15개중에서 제거 단어 수
위해
친구
아버지
엄마
가족
5 개


In [30]:
# 제거
filtered_w1 = [w for w in w1 if vectorizer.get_feature_names_out()[w[0]] not in remove_from_art]
filtered_w2 = [w for w in w2 if vectorizer.get_feature_names_out()[w[0]] not in remove_from_gen]

# TF-IDF 점수대로 정렬
filtered_w1.sort(key=lambda x: x[1], reverse=True)
filtered_w2.sort(key=lambda x: x[1], reverse=True)

In [31]:
# 단어 리스트 변환
filtered_family_words = [vectorizer.get_feature_names_out()[w[0]] for w in filtered_w1[:100]]
filtered_drama_words = [vectorizer.get_feature_names_out()[w[0]] for w in filtered_w2[:100]]

In [34]:
n = 15
target_family, target_drama = [], []

for word in filtered_family_words:
    if word in model.wv:  # Word2Vec에 있는 단어만 사용
        target_family.append(word)
    if len(target_family) == n:
        break

for word in filtered_drama_words:
    if word in model.wv:
        target_drama.append(word)
    if len(target_drama) == n:
        break

print("Family 단어 15개:", target_family)
print("Drama 단어 15개:", target_drama)

Family 단어 15개: ['엄마', '아빠', '가족', '위해', '친구', '아버지', '아들', '마을', '아이', '할머니', '학교', '세상', '소년', '가장', '어머니']
Drama 단어 15개: ['자신', '영화제', '그녀', '사람', '사랑', '영화', '시작', '국제', '남자', '이야기', '여자', '대한', '마음', '단편', '서울']


In [33]:
### 이전과 비교 ###
# ['아주르', '혼자', '낙타', '모험', '위기', '씨제이', '엠마', '할아버지', '미아', '동구', '고양이', '시골', '크리스마스', '소식', '손녀']
# ['남자', '여자', '남편', '감독', '연출', '관계', '생활', '결혼', '우리', '대해', '여성', '죽음', '부문', '세계', '현실']

공통적으로 중요한 경우도 있으며, TF-IDF 점수가 높다고 해서 특정한 카테고리를 대표하는 단어라고 보장할 수 없다. 하지만 빈도기반인 TF-IDF에서 해당 방법이 단순 제거보다 효율적으로 보인다.

### 4. embedding model과 단어 셋으로 WEAT score 구해보기

In [46]:
X = vectorizer.fit_transform(genre) # Family & Drama
print(X.shape)

(2, 24276)


In [35]:
genre_name

['가족', '드라마']

In [48]:
m = [X[i].tocoo() for i in range(X.shape[0])]

w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)
attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) < 15):
        if vectorizer.get_feature_names()[w[i][j][0]] in model.wv:
            attr.append(vectorizer.get_feature_names()[w[i][j][0]])
            print(vectorizer.get_feature_names()[w[i][j][0]], end=', ')
        j += 1
    attributes.append(attr)
    print()

가족: 엄마, 영화제, 아빠, 가족, 자신, 위해, 친구, 아버지, 시작, 그녀, 아들, 마을, 국제, 사람, 아이, 
드라마: 자신, 영화제, 그녀, 사람, 사랑, 위해, 영화, 시작, 국제, 남자, 친구, 이야기, 아버지, 엄마, 가족, 


In [49]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))

def s(w, A, B):
    c_a = cos_sim(w, A)
    c_b = cos_sim(w, B)
    mean_A = np.mean(c_a, axis=-1)
    mean_B = np.mean(c_b, axis=-1)
    return mean_A - mean_B #, c_a, c_b

def weat_score(X, Y, A, B):
    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

### WEAT 비교

#### 1. 중복 모두 제거

In [50]:
print(target_art)
print(target_gen)

['아주르', '혼자', '낙타', '모험', '위기', '씨제이', '엠마', '할아버지', '미아', '동구', '고양이', '시골', '크리스마스', '소식', '손녀']
['남자', '여자', '남편', '감독', '연출', '관계', '생활', '결혼', '우리', '대해', '여성', '죽음', '부문', '세계', '현실']


In [52]:
X = np.array([model.wv[word] for word in target_art])
Y = np.array([model.wv[word] for word in target_gen])

matrix = [[0 for _ in range(len(genre_name))] for _ in range(len(genre_name))]

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model.wv[word] for word in attributes[i]])
        B = np.array([model.wv[word] for word in attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)

In [53]:
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print(genre_name[i], genre_name[j],matrix[i][j])

가족 드라마 1.4275923


#### 2. 중복 단어를 가장 score가 높은곳에 할당할 경우

In [104]:
print(target_family)
print(target_drama)

['엄마', '아빠', '가족', '위해', '친구', '아버지', '아들', '마을', '아이', '할머니', '학교', '세상', '소년', '가장', '어머니']
['자신', '영화제', '그녀', '사람', '사랑', '영화', '시작', '국제', '남자', '이야기', '여자', '대한', '마음', '단편', '서울']


In [105]:
X = np.array([model.wv[word] for word in target_family])
Y = np.array([model.wv[word] for word in target_drama])

matrix = [[0 for _ in range(len(genre_name))] for _ in range(len(genre_name))]

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model.wv[word] for word in attributes[i]])
        B = np.array([model.wv[word] for word in attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)

In [106]:
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print(genre_name[i], genre_name[j],matrix[i][j])

가족 드라마 1.6726209


주요 단어 선정에 있어 단순 중복 제거보다 조금 개선된 부분이 있는 것처럼 보이지만, TF-IDF 방식 사용시 더 정교한 처리가 필요해 보인다.